# Downloading the full dataset as a spreadsheet (XLSX) and analysing with Pandas

As we saw, when your search results in more than 10 000 organisations, it is not possible to get access to all of them through the search-API. Instead it is recommended to download the full dataset. This could be done in many ways, for instance through manually download on the [home page of the open data from the registry](https://data.brreg.no/enhetsregisteret/oppslag/enheter), or using curl as shown in the [API-documentation](https://data.brreg.no/enhetsregisteret/api/docs/index.html#enheter-lastned). Below, it is shown how to use Python to download and save the file, as well as how to read and analyse the file with Pandas.

Pros:
- already "flattened" and easy to work with for example with Python Pandas
- the result-file is smaller than the JSON-file

Cons:
- normally requires to read the full file in memory before manipulating it/making a new file with a subset of the data
- harder to combine with data from the API (different names for columns/elements)

As with the search-API, We use requests for getting data from the server

In [ ]:
!pip install requests

In [ ]:
import requests

Since the size of the file is quite large, we will use a method where the file is downloaded by Python and written to a local file in chunks, instead of Python having to download and open the full file, before it writes it to a local file. Be aware that the files are delivered as compressed files, that can not be opened directly in an editor. One solution is to uncompress the file after it is downloaded, but as we will see, it is also possible for Python to use the compressed file directly when we use the Pandas-library.

In [ ]:
# Laster ned XLSX:
url = 'https://data.brreg.no/enhetsregisteret/api/enheter/lastned/regneark'
headers = {'Accept': 'application/vnd.brreg.enhetsregisteret.enhet+vnd.openxmlformats-officedocument.spreadsheetml.sheet;charset=UTF-8'}
session = requests.Session() # establish a session that is kept open during the transfer, instead of performing separate requests
r = session.get(url, headers=headers, stream = True)
r.raise_for_status()
with open('er.xlsx','wb') as f:
    for chunk in r.iter_content(1024*1024*10): # approx 10 MB in each chunck
        f.write(chunk)

## Part 3: Opening and analysing the file with Pandas
Having downloaded the file, as we saw in the above section, is of little use if we don't get access to it. A good tool for opening large data files and analysing them, is Pandas. Pandas has method for many file types, and also handles the fact that they are compressed (gzip).

In [ ]:
!pip install pandas

The most common way to import Pandas, and numpy which is often necessary as it contains relevant data-types:

In [ ]:
import pandas as pd
import numpy as np

In some cases, you also need to explicitly install openpyxl for pandas to be able to read files of type xlsx

In [ ]:
!pip install openpyxl

We could just let pandas read and guess the datatype (dtype) of each column, but it saves us some time to have it specified already at the import. And now that the code is here, you don't have to type it manually ... :-)

In [ ]:
df_new = pd.read_excel('er.xlsx', dtype={
        'Organisasjonsnummer': str,
        'Navn': str,
        'Organisasjonsform.kode': 'category',
        'Organisasjonsform.beskrivelse': 'category',
        'Næringskode 1': str,
        'Næringskode 1.beskrivelse': str,
        'Næringskode 2': str,
        'Næringskode 2.beskrivelse': str,
        'Næringskode 3': str,
        'Næringskode 3.beskrivelse': str,
        'Hjelpeenhetskode': 'category',
        'Hjelpeenhetskode.beskrivelse': 'category',
        'Antall ansatte': np.int16,
        'Hjemmeside': str,
        'Postadresse.adresse': str,
        'Postadresse.poststed': str,
        'Postadresse.postnummer': str,
        'Postadresse.kommune': str,
        'Postadresse.kommunenummer': str,
        'Postadresse.land': 'category',
        'Postadresse.landkode': 'category',
        'Forretningsadresse.adresse': str,
        'Forretningsadresse.poststed': str,
        'Forretningsadresse.postnummer': str,
        'Forretningsadresse.kommune': str,
        'Forretningsadresse.kommunenummer': str,
        'Forretningsadresse.land': 'category',
        'Forretningsadresse.landkode': 'category',
        'Institusjonell sektorkode': 'category',
        'Institusjonell sektorkode.beskrivelse': 'category',
        'Siste innsendte årsregnskap': str, # klarte ikke konvertere til np.int16
        'Registreringsdato i Enhetsregisteret': 'datetime64',
        'Stiftelsesdato': str, # klarte ikke å konvertere til datetime64 - 1550-12-31 00:00:00
        'FrivilligRegistrertIMvaregisteret': 'category',
        'Registrert i MVA-registeret': 'category',
        'Registrert i Frivillighetsregisteret': 'category',
        'Registrert i Foretaksregisteret': 'category',
        'Registrert i Stiftelsesregisteret': 'category',
        'Konkurs': 'category',
        'Under avvikling': 'category',
        'Under tvangsavvikling eller tvangsoppløsning': 'category',
        'Overordnet enhet i offentlig sektor': str,
        'Målform': 'category' })

Now we can get som info about the dataset

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.sample(10)

When exploring and setting up the relevant analysis, you could consider making a sample of the dataset, if it is slow to work with the full registry. Using frac=0.2 gives you a random sample of 20 % of the data.

In [ ]:
df_sample = df.sample(frac=0.2)

In [ ]:
len(df_sample)

It might also be nice to save the sample as a file so that you don't need to wait for Python to read the full dataset the next time you work on your project. 

In [ ]:
df_sample.to